In [1]:
import requests
import json
import os
import climate
from pprint import pprint
import pandas as pd
import numpy as np
import io
import census
from pprint import pprint

In [3]:
ca = census.CensusData()
co = climate.ClimateAgent()
pgpw = os.getenv("POSTGRES_PASSWORD") 
server, engine = ca.connect_to_postgres(password=pgpw) #need to add create=True if


In [ ]:
# ca.upload_city_data_to_postgres(tmp_df, engine=engine)
# # CITY YEAR will be the primary key(s)
# # all other data will follow after that

In [ ]:
# # to delete a table,,, weird syntax
# from sqlalchemy import MetaData, Table
# metadata = MetaData()

# # Reflect the existing table
# table_name = 'city_data'
# table = Table(table_name, metadata, autoload_with=engine)

# # Drop the table
# table.drop(engine)

In [ ]:
# query = """
# SELECT * 
# FROM cities
# """

# counties = list(pd.read_sql_query(query, con=engine)['county'])

# final_df = pd.DataFrame()
# for county in counties:
#     print("Working on: ", county)
#     city_data_df = ca.get_data_for_city(city=None, years=(2009,2024), county=county)
#     final_df = pd.concat([final_df, city_data_df], ignore_index=True)


In [ ]:
# ca.upload_city_data_to_postgres(final_df, engine=engine)

In [ ]:
# # existing_data = pd.read_sql('SELECT city, year FROM city_data', engine)
# # new_rows = final_df[~final_df[['city', 'year']].isin(existing_data[['city', 'year']])]
# # final_df[['city', 'year']].isin(existing_data[['city', 'year']])
# query = """
# SELECT * FROM city_data
# """

# check_df = pd.read_sql_query(query, con=engine)
# check_df
# #rows_to_upload = final_df[]
# # final_df.to_sql("city_data", con=engine, index=False, chunksize=1000, if_exists="replace")

In [ ]:

# query = """
# SELECT * FROM city_data
# """

# df = pd.read_sql_query(query, con=engine)
# df

In [ ]:
# y = "2009"
# url = "https://api.census.gov/data/" + f"{y}/acs/acs1/profile"

# get_metrics = "".join(x + "," for x in ca.list_of_metrics.keys())
# get_metrics = get_metrics[:len(get_metrics)-1] # get rid of the last comma that I appended

# params = {"key": os.getenv("CENSUS_KEY"),
#                     "get": get_metrics,
#                     "for": f"county:810",
#                     "in": "state:51"}

# r = requests.get(url=url, params=params)
# values = list(json.loads(r.text))
# pd.DataFrame(values[1:], columns=values[0])

In [ ]:
# ca.get_all_cities()

In [ ]:
# from html.parser import HTMLParser
# #census_url = "https://api.census.gov/data"
# year = "2023"
# dataset = "acs"
# url = f"https://api.census.gov/data/{year}/acs/acs1/profile"


# """
#     # https://api.census.gov/data/2022/acs/acs1/profile/variables.html 

#     # ones worth using:

#     # economic
#     DP03_0093E = estimate for earnings for first year male workers 
#     DP03_0094E = estimate for earnings for first year female workers
#     DP03_0119PE = estimated percentage of people below the poverty line
#     DP04_0001E = estimated housing units
#     DP02_0002E = estimated number of married housing units
#     DP02_0016E = estimated average household size

#     DP03_0052E: estimated total households less than $10,000 income
#     DP03_0053E: estimated total households between $10-15k
#     DP03_0054E: estimated total households between $15-25k
#     DP03_0055E: 25-35k
#     DP03_0056E: 35-45k
#     DP03_0057E: 50-75k
#     DP03_0058E: 75-100k
#     DP03_0059E: 100-150k
#     DP03_0060E: 150-200k
#     DP03_0061E: > 200k

#     # relationships
#     DP02_0026E = men over 15 y/o who have not married
#     DP02_0028E = men over 15 y/o who married, but separated
#     DP02_0030E = men over 15 y/o who are divorced
    
#     # education (currently)
#     DP02_0054E = estimated population of kids in nursery school/preschool
#     DP02_0055E = estimated population of kids in kindergarten
#     DP02_0056E = estimated population of kids in grades 1-8
#     DP02_0057E = estimated population of kids in grades 9-12
#     DP02_0058E = estimated population of kids in college/grad school

#     # education attainment
#     DP02_0060PE = estimated population of 25 years and older who have less than 9th grade education
#     DP02_0061PE = estimated population with some high school education
#     DP02_0062PE = estimated population with high school diploma
#     DP02_0063PE = estimated population with some college, no diploma
#     DP02_0064PE = estimated population with associates degrees
#     DP02_0065PE  = estimated population with bachelor's degree
#     DP02_0066PE = estimated population with graduate degrees

#     # unemployement rate
#     DP03_0009PE

#     # industry
#     DP03_0033PE: agriculture, forestry, fishing, hunting, mining
#     DP03_0034PE: construction
#     DP03_0035PE: manufacturing
#     DP03_0036PE: wholesale trade
#     DP03_0037PE: retail trade
#     DP03_0038PE: transportation, warehousing, utilities
#     DP03_0039PE: information
#     DP03_0040PE: finance, insurance, real estate, rental, leasing
#     DP03_0041PE: professional, scientific, administrative, waste management
#     DP03_0042PE: educational services/health care/ social assistance
#     DP03_0043PE: arts, entertainment, recreation, accommodation, food services
#     DP03_0045PE: public administration

# """
# variables = "NAME,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE"
# print(url)
# params = {"key": os.getenv("CENSUS_KEY"),
#            "for": "county:*", 
#            "get": variables,
#            "in": "state:51"
#           }
# r = requests.get(url, params=params)
# values = list(json.loads(r.text))

# df = pd.DataFrame(values[1:], columns=values[0])
# # df = df.rename(columns={
# #     "DP02_0060PE":"<9th",
# #     "DP02_0061PE": "some high school",
# #     "DP02_0062PE": "high school",
# #     "DP02_0063PE": "some college",
# #     "DP02_0064PE": "associates",
# #     "DP02_0065PE": "bachelors",
# #     "DP02_0066PE": "graduate"
# # })
# df

In [ ]:
co = climate.ClimateAgent()
# let's test for all of the cities that we have data on for the Census
# so there will be around 30 "cities" (counties, really) for Virginia that we will have data on
cities = ca.get_all_cities()
cities = list(cities['city'])
for c in cities:
    print("Working on:" c)
    station_id = co.get_station_id(c)
    climate_df = co.get_data_for_station_id(station_id=station_id, start_date="2022-01-01", end_date="2022-12-31")

In [ ]:
station_id = co.get_station_id("Norfolk, Virginia")
climate_df = co.get_data_for_station_id(station_id=station_id, start_date="1930-01-01", end_date="2022-12-31")

In [ ]:
co.upload_data_to_postgres(climate_data=climate_df, engine=engine)

In [ ]:
query = """
SELECT *
FROM climate_data
"""

pd.read_sql_query(query, con=engine)

In [23]:
cities = ca.get_all_cities()
cities = list(cities['city'])

city_to_station = {}

for c in cities:
    if c == "fauquier":
        c = "front royal"
    if c == "jamescity":
        c = "williamsburg"
    if c == "princewilliam":
        c = "manassas"
    c = c + ", virginia"
    print("Working on:", c)
    city_id = co.get_station_id(city=c)
    city_to_station[c] = city_id
pprint(city_to_station)

Working on: albemarle, virginia
Working on: arlington, virginia
Working on: augusta, virginia
Working on: bedford, virginia
Working on: chesterfield, virginia
Working on: fairfax, virginia
Working on: front royal, virginia
Working on: frederick, virginia
Working on: hanover, virginia
Working on: henrico, virginia
Working on: williamsburg, virginia
Working on: loudoun, virginia
Working on: montgomery, virginia
Working on: manassas, virginia
Working on: roanoke, virginia
Working on: rockingham, virginia
Working on: spotsylvania, virginia
Station id look up failed for: spotsylvania, virginia
{"dataTypes":{"docCountError":0,"buckets":[],"sumOfOtherDocCounts":0},"count":0,"bounds":{"bottomRight":null,"topLeft":null},"totalFileSize":0.0,"stations":{"docCountError":0,"buckets":[],"sumOfOtherDocCounts":0},"totalCount":123848,"results":[]}


UnboundLocalError: cannot access local variable 'station_id' where it is not associated with a value

In [14]:
print(cities)

['albemarle', 'arlington', 'augusta', 'bedford', 'chesterfield', 'fairfax', 'fauquier', 'frederick', 'hanover', 'henrico', 'jamescity', 'loudoun', 'montgomery', 'princewilliam', 'roanoke', 'rockingham', 'spotsylvania', 'stafford', 'york', 'lynchburg', 'alexandria', 'chesapeake', 'hampton', 'newportnews', 'norfolk', 'portsmouth', 'richmond', 'roanoke', 'suffolk', 'beach']


In [19]:
lat, long = co.get_lat_long(location="manassas VA")
bb = co.get_bounding_box(latitude=lat, longitude=long, buffer = 10)
bb

(38.896, -77.661, 38.606, -77.289)

In [20]:
bounding_box = f"{bb[0]},{bb[1]},{bb[2]},{bb[3]}"
print(bounding_box)
params = {"dataset":"global-summary-of-the-month",
                  "bbox": bounding_box,
                  "dataTypes": "TMIN,TMAX,PRCP",
                  "startDate": "2022-01-01",
                  "endDate": "2022-02-01"}
headers = co.make_headers()
base = "https://www.ncei.noaa.gov/access/services/search/v1/data"
r = requests.get(url=base, params=params, headers=headers)
r.text

38.896,-77.661,38.606,-77.289


'{"dataTypes":{"docCountError":0,"buckets":[{"docCount":2,"key":"CDSD"},{"docCount":2,"key":"CLDD"},{"docCount":2,"key":"DP01"},{"docCount":2,"key":"DP10"},{"docCount":2,"key":"DP1X"},{"docCount":2,"key":"DSND"},{"docCount":2,"key":"DSNW"},{"docCount":2,"key":"DT00"},{"docCount":2,"key":"DT32"},{"docCount":2,"key":"DX32"},{"docCount":2,"key":"DX70"},{"docCount":2,"key":"DX90"},{"docCount":2,"key":"DYFG"},{"docCount":2,"key":"DYNT"},{"docCount":2,"key":"DYSD"},{"docCount":2,"key":"DYSN"},{"docCount":2,"key":"DYTS"},{"docCount":2,"key":"DYXP"},{"docCount":2,"key":"DYXT"},{"docCount":2,"key":"EMNT"},{"docCount":2,"key":"EMSD"},{"docCount":2,"key":"EMSN"},{"docCount":2,"key":"EMXP"},{"docCount":2,"key":"EMXT"},{"docCount":2,"key":"HDSD"},{"docCount":2,"key":"HTDD"},{"docCount":2,"key":"PRCP"},{"docCount":2,"key":"SNOW"},{"docCount":2,"key":"TAVG"},{"docCount":2,"key":"TMAX"},{"docCount":2,"key":"TMIN"}],"sumOfOtherDocCounts":0},"endDate":"2024-09-30T23:59:59","count":2,"bounds":{"bottomRig